## groupBy()

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
schema = StructType([
    StructField('class_cd', StringType()),
    StructField('school', StringType()),
    StructField('class_std_cnt', IntegerType()),
    StructField('loc', StringType()),
    StructField('school_type', StringType()),
    StructField('teaching_type', StringType()),
])

cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True, schema=schema)
cdf.printSchema()
cdf.createOrReplaceTempView('class')

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental|
|     PC6| CIMBB|           17|   Urban| Non-public|     Stand

In [28]:
print('지역별 교육타입별 학생 숫자를 구해보자.')
cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt)) \
.show(5)
# 평균도 같이 구해보자
cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt), avg('class_std_cnt')) \
.show(5)

print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단,  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')
cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt)>= 300) \
.show(5)

# ddf.select('loc', 'teaching_type', ddf.(sum(class_std_cnt) >= 300).alias('total'))show(5)

# 접근은 나쁘지 않았으나 조건을 달기 전에 먼저 별칭을 지정해 줘야 했음......

# 별칭을 지정해서 진행
tmp = cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt).alias('total'), avg('class_std_cnt').alias('avg'))
tmp.show(5)
tmp.where(col('total') >= 300).show(5)
tmp.where(tmp.total>= 300).show(5)

print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')

print('지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.')

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자.')

print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')


지역별 교육타입별 학생 숫자를 구해보자.
+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
+--------+-------------+------------------+
only showing top 5 rows

+--------+-------------+------------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|avg(class_std_cnt)|
+--------+-------------+------------------+------------------+
|   Rural| Experimental|               211|17.583333333333332|
|    null|         null|              null|              null|
|   Urban|     Standard|               631| 24.26923076923077|
|Suburban|     Standard|               433|             21.65|
|   Rural|     Standard|               327|           20.4375|
+--------+-------------+------------------+---------

In [45]:
# 반이 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
# base = cdf \
#                 .where(cdf.loc.isNotNull())
#                 .groupBy(cdf.loc) \
#                 .agg(count(cdf.class_cd).alias('cnt'), sum(cdf.class_std_cnt).alias('tot')) \
#                 .select(max('cnt'), min('cnt'))

base = cdf.groupBy(cdf.loc) \
        .agg(count(cdf.class_cd).alias('cnt')
             , sum(cdf.class_std_cnt).alias('tot')) \
        .where(cdf.loc.isNotNull())
base.show()
base.collect()

min_max_row = base.select(max('cnt'), min('cnt')).collect()
min_max_row

base.where(base.cnt.isin(min_max_row[0][0], min_max_row[0][1])) \
    .select(max(col('tot')) - min(col('tot'))).show()


+--------+---+---+
|     loc|cnt|tot|
+--------+---+---+
|   Urban| 37|906|
|Suburban| 34|717|
|   Rural| 28|538|
+--------+---+---+



[Row(loc='Urban', cnt=37, tot=906),
 Row(loc='Suburban', cnt=34, tot=717),
 Row(loc='Rural', cnt=28, tot=538)]

[Row(max(cnt)=37, min(cnt)=28)]

+---------------------+
|(max(tot) - min(tot))|
+---------------------+
|                  368|
+---------------------+



#### sql

In [9]:
cdf.printSchema()

print('지역별 교육타입별 학생 숫자를 구해보자.')
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt) from class group by loc, teaching_type
''').show()

print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단,  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt) as total, avg(class_std_cnt) as avg
    from class
    group by loc, teaching_type
    having sum(class_std_cnt) >= 300
''').show(5)

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')

print('지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.')

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자.')

print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')


root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

지역별 교육타입별 학생 숫자를 구해보자.
+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+

지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단,  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.
+--------+-------------+-----+-----------------+
|     loc|teaching_type|total|              avg|
+--------+-------------+-----+-----------------+
|   Urban|     Stand

In [ ]:
# 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자





## orderBy()

In [43]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
# 방법1 (권장)
cdf.select('*') \
    .orderBy(cdf.class_std_cnt.desc()).show(5)
# 방법 2
cdf.select('*') \
    .orderBy(cdf.class_std_cnt, ascending= False).show(5)
# 방법 3
cdf.select('*') \
    .orderBy(cdf.class_std_cnt, ascending=[0]).show(5)

print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
# 방법 1 (권장)
cdf.select('*') \
    .orderBy(cdf.loc.asc(), cdf.school.desc()).show(50)
# 방법 2
cdf.select('*') \
    .orderBy(cdf.loc.asc(), desc('school')).show(50)
# 방법 3
cdf.select('*') \
    .orderBy(["loc", "school"], ascending=[0, 1]).show(50)

print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
cdf.select('*') \
    .orderBy(cdf.school_type.asc_nulls_first()).show(30)

### 앞으로 select('*')을 하려면 생략가능 cdf.select('*').orderBy => cdf.orderBy

반 학생 숫자를 기준으로 내림차순 정렬하라
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     QA2| ZOWMK|           30|Urban|     Public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 5 rows

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Exper

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     PC6| CIMBB|           17|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental|
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     PBA| GOOBU|           24|   Urban|     Public|     Standard|
|     U6J| GOOBU|           25|   Urban|     Public|     Stand

#### sql

In [26]:
cdf.printSchema()

print('반 학생 숫자를 기준으로 내림차순 정렬하라')
spark.sql('''
    select * from class order by class_std_cnt desc
''').show(5)


print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql('''
    select * from class order by loc asc, school desc
''').show(10)


print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
    select * from class order by school_type asc nulls first
''').show(10)
spark.sql('''
    select * from class order by school_type asc nulls last
''').show(10)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

반 학생 숫자를 기준으로 내림차순 정렬하라
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     QA2| ZOWMK|           30|Urban|     Public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 5 rows

loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라
+--------+------+-------------+-----+-----------+-------------+
|cla